# Prétraitement des données de Vie-publique

### Importation des librairies

In [1]:
import pandas as pd
import locale
import re 

### Importation des csv en df

In [2]:
scraping_df1 = pd.read_csv('Viepublique1.csv')

In [3]:
scraping_df2 = pd.read_csv('Viepublique2.csv')

In [4]:
scraping_df3 = pd.read_csv('Viepublique3.csv')

In [5]:
scraping_df4 = pd.read_csv('Viepublique4.csv')

In [6]:
scraping_df5 = pd.read_csv('Viepublique5.csv')

### Concaténation des csv

In [7]:
df_merged = pd.concat([scraping_df1, scraping_df2, scraping_df3, scraping_df4, scraping_df5], ignore_index=True, sort=False)

### Suppression des duplicats

Les duplicats sont causés par la manière de récupérer les données, pas des données en elles-mêmes

In [8]:
df_vie_publique = df_merged.drop_duplicates(subset = "link")

In [9]:
df_vie_publique['Texte'].isna().sum() #nombre d'interview vides (sans texte)

np.int64(15700)

### Suppression des interviews vides

In [10]:
df_vie_publique.dropna(subset = ['Texte'], inplace=True)

/var/folders/bf/dpp1hl0x74gg20m3v0lh7p_r0000gn/T/ipykernel_960/958617740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vie_publique.dropna(subset = ['Texte'], inplace=True)


### Suppression de "Prononcé le \n" dans la colonne Date

In [11]:
df_vie_publique['Date'] = df_vie_publique['Date'].str.replace('Prononcé le\n', '', regex=False)

/var/folders/bf/dpp1hl0x74gg20m3v0lh7p_r0000gn/T/ipykernel_960/3416638251.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vie_publique['Date'] = df_vie_publique['Date'].str.replace('Prononcé le\n', '', regex=False)


### Conversion des dates en string

In [12]:
df_vie_publique['Date'] = df_vie_publique['Date'].astype(str) # mettre la colonne date en str (car elle est de type objet pour l'instant)

/var/folders/bf/dpp1hl0x74gg20m3v0lh7p_r0000gn/T/ipykernel_960/2626484436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vie_publique['Date'] = df_vie_publique['Date'].astype(str) # mettre la colonne date en str (car elle est de type objet pour l'instant)


In [13]:
# Nettoyer les espaces avant et après les chaînes de caractères
df_vie_publique['Date'] = df_vie_publique['Date'].str.strip()

/var/folders/bf/dpp1hl0x74gg20m3v0lh7p_r0000gn/T/ipykernel_960/3009945388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vie_publique['Date'] = df_vie_publique['Date'].str.strip()


### Remplacer dans les dates "1er" par "1" car cela empeche la converstion en format "date"

In [14]:
# le format n'est pas bon pour ces données car il y a "er" après le 1 alors qu'il ne faudrait pas
# on va donc remplacer chaque "1er" qui apparait dans ma colonne date par un "1"

In [15]:
df_vie_publique['Date'] = df_vie_publique['Date'].str.replace('1er', '1', regex=False)

/var/folders/bf/dpp1hl0x74gg20m3v0lh7p_r0000gn/T/ipykernel_960/2846919873.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vie_publique['Date'] = df_vie_publique['Date'].str.replace('1er', '1', regex=False)


### Conversion des dates en un format spécifique pour les dates

In [16]:
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8') 
df_vie_publique['Date'] = pd.to_datetime(df_vie_publique['Date'], format='%d %B %Y', errors='coerce')

/var/folders/bf/dpp1hl0x74gg20m3v0lh7p_r0000gn/T/ipykernel_960/1772832154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vie_publique['Date'] = pd.to_datetime(df_vie_publique['Date'], format='%d %B %Y', errors='coerce')


### Garder uniquement les textes commençants par "Texte intégral"

In [17]:
patternDel = "^Texte intégral.*"
df_filter = df_vie_publique['Texte'].str.contains(patternDel)
df_vie_publique = df_vie_publique[df_filter]

### Nettoyer la colonne "intervenants_metiers"

Suppression de l'indication du metier car il n'est pas normalisé

In [27]:
df_vie_publique['intervenants_metiers'] = df_vie_publique['intervenants_metiers'].str.replace(r'\n.*', '', regex=True)

In [28]:
# Compter le nombre d'occurrences de chaque nom
df_vie_publique['intervenants_metiers'].value_counts()

intervenants_metiers
François Bayrou        527
François Mitterrand    522
Hubert Védrine         522
Laurent Fabius         483
Bernard Kouchner       410
                      ... 
Janos Martonyi           1
Monique Vuaillat         1
Julia Ficatier           1
Robin Cook               1
Edith Cresson            1
Name: count, Length: 617, dtype: int64

### Supprimer "Texte intégral" en début de chaque texte

In [29]:
df_vie_publique['Texte'] = df_vie_publique['Texte'].str.replace(r'Texte intégral\s*\n\s*\n', '', regex=True)

In [30]:
df_vie_publique['Texte'] = df_vie_publique['Texte'].str.replace('Texte intégral\n          \n            ', '', regex=False)

### Nettoyer la colonne "interviewer_metier"

Nous souhaitons uniquement garder le nom du journaliste et pas le nom du metier qui lui est associé (qui est presque toujours "journaliste")

In [31]:
#On enlève tout ce qui est après \n dans chaque ligne de la colonne "interviewer_metier"
df_vie_publique['interviewer_metier'] = df_vie_publique['interviewer_metier'].str.replace(r'\n.*', '', regex=True)

In [32]:
# compte du nombre d'interviews pas journalistes
df_vie_publique['interviewer_metier'].value_counts()

interviewer_metier
Jean-Pierre Elkabbach    953
Jean-Jacques Bourdin     787
Jean-Michel Aphatie      573
Olivier Mazerolle        382
Françoise Laborde        379
                        ... 
Bachir Rezzoug             1
Peter Scholl-Latour        1
Francesco Colombo          1
Barack Obama               1
Linda Giger                1
Name: count, Length: 1886, dtype: int64

#### On enlève la partie du texte qui mentionne la source dans la colonne "Texte"

In [33]:
# Suppression du texte après le dernier point suivi de "Source"
df_vie_publique['Texte'] = df_vie_publique['Texte'].str.replace(r'\.[^.]*Source.*$', '.', regex=True)

### Suppression des colonnes qui ne nous intéressent pas

In [35]:
df_vie_publique = df_vie_publique.drop(['Topic', 'mots_cles'], axis=1)

### Convertir le df "df_vie_publique" en csv

In [36]:
df_vie_publique.to_csv("csv_vie_publique.csv", encoding= 'utf-8' )